In [1]:
import sys
sys.path.append('..')
import os
from common.np import *

# 2.1 単語の意味の表現方法
- シソーラス  
類語辞書．人の手で単語同士の関連を定義する．


- カウントベース  
「単語の意味は周囲の単語によって形成される」という分布仮説に基づく．
大量のテキストデータを使い，ある単語の周囲の単語をカウントすることで，単語をベクトルで表現する．


- 推論ベース  
分布仮説に基づく．
ある単語の周囲の単語を特徴量として，単語を推測するモデルを使う．

# 2.3 カウントベースの手法

## コーパスの下準備
以下の3つを行う preprocess関数を実装
- コーパスを単語IDのリストに変換
- IDから単語へ変換するためのディクショナリを用意
- 単語からIDへ変換するためのディクショナリを用意

In [2]:
def preprocess(text):
    
    text = text.lower()             # 文字を小文字に変える
    text = text.replace('.', ' .')  # ピリオドの前にスペースを入れる
    words = text.split(' ')         # スペースで分割してリスト化

    word_to_id = {}  # 単語からIDに変換するためのディクショナリ
    id_to_word = {}  # IDから単語に変換するためのディクショナリ
    
    # ディクショナリにない単語の場合，新しいIDとして追加する
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    # リスト化した単語をIDに変換する
    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [3]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [4]:
print('text      :', text)
print('corpus    :', corpus)
print('word_to_id:', word_to_id)
print('id_to_word:', id_to_word)

text      : You say goodbye and I say hello.
corpus    : [0 1 2 3 4 1 5 6]
word_to_id: {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
id_to_word: {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


## 共起行列
- 行: 単語を表すベクトル
- 列: その単語がwindow内に何度出現したかをカウントした値

In [5]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''共起行列の作成

    :param corpus: コーパス（単語IDのリスト）
    :param vocab_size:語彙数
    :param window_size:ウィンドウサイズ（ウィンドウサイズが1のときは、単語の左右1単語がコンテキスト）
    :return: 共起行列
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    # ループを回して，corpusの単語ごとに処理
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            # コンテキストのインデックスを取得
            left_idx = idx - i
            right_idx = idx + i

            # 左側のコンテキストがcorpus内であればco_matrixに値を追加
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            # 右側のコンテキストがcorpus内であればco_matrixに値を追加
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [6]:
# 共起行列を作る
# vocab_sizeは辞書の要素数
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
print(C)

[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


### コサイン類似度

$$
\mathrm{similarity}(\boldsymbol{x}, \boldsymbol{y})
= \frac{\boldsymbol{x} \cdot \boldsymbol{y}}{\| \boldsymbol{x} \| \| \boldsymbol{y} \|}
= \frac{x_1 y_1 + \cdots + x_n y_n}{\sqrt{x_1^2 + \cdots + x_n^2} \sqrt{y_1^2 + \cdots + y_n^2}}
\\
\\
\boldsymbol{x} \cdot \boldsymbol{y} = \| \boldsymbol{x} \| \| \boldsymbol{y} \| \mathrm{cos \theta}
$$

- ベクトルの類似度の指標として使う．
- 内積をノルムの積で割るので結局は $\mathrm{cos} \theta$．$\theta$ はベクトル $\boldsymbol{x}, \boldsymbol{y}$ の成す角．
- 直交するとき $\mathrm{similarity}(\boldsymbol{x}, \boldsymbol{y}) = 0$．
- 同じ方向を向くとき $\mathrm{similarity}(\boldsymbol{x}, \boldsymbol{y}) = 1$．
- 反対方向を向くとき $\mathrm{similarity}(\boldsymbol{x}, \boldsymbol{y}) = -1$

In [7]:
def cos_similarity(x, y, eps=1e-8):
    '''コサイン類似度の算出

    :param x: ベクトル
    :param y: ベクトル
    :param eps: ”0割り”防止のための微小値
    :return:
    '''
    # ノルムを 1 にしてから内積を取る
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [8]:
# you と i のコサイン類似度を求めてみる
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))

0.7071067691154799


# 類似単語のランキング表示
- クエリとして与えられた単語に対して，コサイン類似度の高い単語を上位から表示する関数を実装

In [11]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''類似単語の検索

    :param query: クエリ（テキスト）
    :param word_to_id: 単語から単語IDへのディクショナリ
    :param id_to_word: 単語IDから単語へのディクショナリ
    :param word_matrix: 単語ベクトルをまとめた行列。各行に対応する単語のベクトルが格納されていることを想定する
    :param top: 上位何位まで表示するか
    '''
    # 辞書に含まれないクエリの場合は処理を実行しない
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    # クエリの単語IDとベクトルを取得
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 語彙数を取得
    vocab_size = len(id_to_word)

    # 類似度を保持する配列を生成
    # 語彙数と同じ要素数が必要
    similarity = np.zeros(vocab_size)
    
    # 共起行列のベクトルを一つずつ取り出し，クエリの単語とのコサイン類似度を計算
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    count = 0
    # 類似度の降順で処理（argsort()では要素の昇順にインデックスを返す）
    for i in (-1 * similarity).argsort():
        # クエリと同じ単語IDのときは処理しない
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [10]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


### argsort()の挙動  
要素の値の昇順にインデックスを並べる  
元の要素に-1をかけると逆順になる

In [17]:
x = np.array([10, -5, 20])
print(x.argsort())
print((-1 * x).argsort())

[1 0 2]
[2 0 1]
